In [1]:
import pandas as pd

In [2]:
def transform(month: int,
              year: int,
              product_code: str              
) -> pd.DataFrame:
    
    """Функция принимает на вход месяц, год и код товара, 
    а возвращает таблицу в формате pd.DataFrame."""

    # Словарь соответствия цифр названиям месяца
    months = {1: 'янв.', 2: 'фев.', 3: 'мар.', 4: 'апр.', 5: 'май.', 6: 'июн.',
              7: 'июл.', 8: 'авг.', 9: 'сен.', 10: 'окт.', 11: 'ноя.', 12: 'дек.'} 

    # Загрузка данных из файла
    data = pd.read_csv("Испытательное.csv", sep=',')

    # Переименование столбцов
    data = data.rename(columns={'Субъект': 'subject',
                                'Код ОКПД-2': 'code',
                                'Округ': 'district',
                                'Месяц': 'month',
                                'Год': 'year',
                                'Значение': 'value'})

    # Выборка данных из таблицы по коду, месяцу и году
    table1 = data.query(f"code == '{product_code}' & month == {month} & year == {year}") \
    .reset_index(drop=True)

    table2 = data.query(f"code == '{product_code}' & month == {month} - 1 & year == {year}") \
        .reset_index(drop=True)

    table3 = data.query(f"code == '{product_code}' & month == {month} - 1 & year == {year} - 1") \
        .reset_index(drop=True)    

    table4 = data.query(f"code == '{product_code}' & year == {year}") \
        .reset_index(drop=True)

    table5 = data.query(f"code == '{product_code}' & year == {year} - 1") \
        .reset_index(drop=True)

    lst = [table1, table2, table3, table4, table5]

    # Преобразование данных (добавление строки с общим количеством, группировка по округу, сортировка)
    for i in range(5):
        lst[i] = lst[i].append({'district': 'Российская Федерация', 
                        'value': lst[i]['value'].sum()}, ignore_index=True)
        lst[i] = lst[i] \
                        .groupby(['district'], as_index=False) \
                        .aggregate({'value': 'sum'}) \
                        .round() \
                        .sort_values('value', ascending=False) \
                        .reset_index(drop=True)

        lst[i]['value'] = lst[i]['value'].astype('Int64')

    # Соединение таблиц с разными датами в одну и рассчет дельты
    final_table = lst[0].merge(lst[1], on='district', how='outer', suffixes=('_1', '_2'))
    final_table['delta1'] = round(((final_table['value_1'] - final_table['value_2']) /
                        ((final_table['value_1'] + final_table['value_2']) / 2) * 100), 1)
    final_table['delta1'] = final_table['delta1'].astype(str) + '%'
    final_table['value'] = final_table['value_1']

    final_table = final_table.merge(lst[2], on='district', how='outer', suffixes=('_3', '_4'))
    final_table['delta2'] = round(((final_table['value_3'] - final_table['value_4']) /
                                ((final_table['value_3'] + final_table['value_4']) / 2) * 100), 1)
    final_table['delta2'] = final_table['delta2'].astype(str) + '%'

    final_table = final_table.merge(lst[3], on='district', how='outer')
    final_table = final_table.merge(lst[4], on='district', how='outer', suffixes=('_5', '_6'))
    final_table['delta3'] = round(((final_table['value_5'] - final_table['value_6']) /
                                ((final_table['value_5'] + final_table['value_6']) / 2) * 100), 1)
    final_table['delta3'] = final_table['delta3'].astype(str) + '%'

    # Конечное переименование столбцов для клиента
    final_table.index.rename(product_code, inplace=True)
    final_table = final_table.rename(columns={'district': 'Регионы РФ',
                                              'value_1': f"{months[month]}{year - 2000} ",
                                              'value_2': f"{months[month - 1]}{year - 2000}",
                                              'delta1': '∆, %',
                                              'value_3': f"{months[month]}{year - 2000}",
                                              'value_4': f"{months[month - 1]}{year - 2001}",
                                              'delta2': '∆, % ',
                                              'value_5': f"c нач.{year}",
                                              'value_6': f"c нач.{year - 1}",
                                              'delta3': '∆, %  '})
 
    # Выделение отрицательных и положительных значений дельты разными цветами
    final_table = final_table \
                            .style \
                            .applymap(func=lambda x: 'color:red' if x[0] == '-' else 'color:green',                                       
                                      subset=['∆, %', '∆, % ', '∆, %  '])
   

    return final_table    

In [9]:
df = transform(6, 2021, '23.52.10.110')
df

,Регионы РФ,июн.21,май.21,"∆, %",июн.21,май.20,"∆, %",c нач.2021,c нач.2020,"∆, %"
23.52.10.110,,,,,,,,,,
0,Российская Федерация,740,720,2.7%,740,715,3.4%,8559,8424,1.6%
1,ЦФО,290,296,-2.0%,290,278,4.2%,3409,3234,5.3%
2,ПФО,198,186,6.2%,198,186,6.2%,2208,2143,3.0%
3,СФО,83,82,1.2%,83,83,0.0%,975,956,2.0%
4,ЮФО,71,52,30.9%,71,50,34.7%,767,626,20.2%
5,УФО,64,71,-10.4%,64,75,-15.8%,783,930,-17.2%
6,СЗФО,25,27,-7.7%,25,41,-48.5%,335,471,-33.7%
7,СКФО,9,7,25.0%,9,1,160.0%,80,62,25.4%
8,ДВФО,0,0,nan%,0,0,nan%,1,2,-66.7%
